# DMRG calculation by ITensors.jl
**Note that this notebook is executed by julia!**

In [1]:
versioninfo()

Julia Version 1.10.4
Commit 48d4fd48430 (2024-06-04 10:41 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M2
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
Threads: 1 default, 0 interactive, 1 GC (on 4 virtual cores)


Depending on your environment (Intel CPU, NVIDIA GPU, etc),
enable following code in the beggining as needed.

    ```julia
    using CUDA
    using MKL
    
    function to_float64_cuda_itensor(itensor_cpu)
        cuarray = CuArray(itensor_cpu.tensor)
        ndtensor = NDTensors.Dense(cuarray)
        itensor_gpu = ITensor(ndtensor, inds(itensor_cpu))
        itensor_gpu
    end
    ```
    After, setting `H`, `psi` ...

    ```julia
    for i in 1:6
        H[i] = to_float64_cuda_itensor(H[i])
    end
    
    for i in 1:6
        psi0_init[i] = to_float64_cuda_itensor(psi0_init[i])
    end
    ```

In [2]:
using ITensors
using HDF5

In [3]:
f = h5open("nnmpo-ham.h5","r")
H = read(f,"H",MPO)
close(f)

In [4]:
H[1] .*= 100 # to reduce numerical error
@show H

H = MPO
[1] ((dim=9|id=646|"Boson,Site,n=1")', (dim=9|id=646|"Boson,Site,n=1"), (dim=7|id=216|"Link,l=1"))
[2] ((dim=9|id=318|"Boson,Site,n=2")', (dim=9|id=318|"Boson,Site,n=2"), (dim=16|id=479|"Link,l=2"), (dim=7|id=216|"Link,l=1"))
[3] ((dim=9|id=818|"Boson,Site,n=3")', (dim=9|id=818|"Boson,Site,n=3"), (dim=16|id=718|"Link,l=3"), (dim=16|id=479|"Link,l=2"))
[4] ((dim=9|id=118|"Boson,Site,n=4")', (dim=9|id=118|"Boson,Site,n=4"), (dim=16|id=741|"Link,l=4"), (dim=16|id=718|"Link,l=3"))
[5] ((dim=9|id=452|"Boson,Site,n=5")', (dim=9|id=452|"Boson,Site,n=5"), (dim=8|id=292|"Link,l=5"), (dim=16|id=741|"Link,l=4"))
[6] ((dim=9|id=2|"Boson,Site,n=6")', (dim=9|id=2|"Boson,Site,n=6"), (dim=8|id=292|"Link,l=5"))



MPO
[1] ((dim=9|id=646|"Boson,Site,n=1")', (dim=9|id=646|"Boson,Site,n=1"), (dim=7|id=216|"Link,l=1"))
[2] ((dim=9|id=318|"Boson,Site,n=2")', (dim=9|id=318|"Boson,Site,n=2"), (dim=16|id=479|"Link,l=2"), (dim=7|id=216|"Link,l=1"))
[3] ((dim=9|id=818|"Boson,Site,n=3")', (dim=9|id=818|"Boson,Site,n=3"), (dim=16|id=718|"Link,l=3"), (dim=16|id=479|"Link,l=2"))
[4] ((dim=9|id=118|"Boson,Site,n=4")', (dim=9|id=118|"Boson,Site,n=4"), (dim=16|id=741|"Link,l=4"), (dim=16|id=718|"Link,l=3"))
[5] ((dim=9|id=452|"Boson,Site,n=5")', (dim=9|id=452|"Boson,Site,n=5"), (dim=8|id=292|"Link,l=5"), (dim=16|id=741|"Link,l=4"))
[6] ((dim=9|id=2|"Boson,Site,n=6")', (dim=9|id=2|"Boson,Site,n=6"), (dim=8|id=292|"Link,l=5"))


In [5]:
truncate!(H; alg="frobenius", cutoff=1.e-15)
@show H

H = MPO
[1] ((dim=9|id=646|"Boson,Site,n=1")', (dim=9|id=646|"Boson,Site,n=1"), (dim=7|id=530|"Link,l=1"))
[2] ((dim=9|id=318|"Boson,Site,n=2")', (dim=9|id=318|"Boson,Site,n=2"), (dim=16|id=694|"Link,l=2"), (dim=7|id=530|"Link,l=1"))
[3] ((dim=9|id=818|"Boson,Site,n=3")', (dim=9|id=818|"Boson,Site,n=3"), (dim=16|id=596|"Link,l=3"), (dim=16|id=694|"Link,l=2"))
[4] ((dim=9|id=118|"Boson,Site,n=4")', (dim=9|id=118|"Boson,Site,n=4"), (dim=16|id=445|"Link,l=4"), (dim=16|id=596|"Link,l=3"))
[5] ((dim=9|id=452|"Boson,Site,n=5")', (dim=9|id=452|"Boson,Site,n=5"), (dim=8|id=927|"Link,l=5"), (dim=16|id=445|"Link,l=4"))
[6] ((dim=9|id=2|"Boson,Site,n=6")', (dim=9|id=2|"Boson,Site,n=6"), (dim=8|id=927|"Link,l=5"))



MPO
[1] ((dim=9|id=646|"Boson,Site,n=1")', (dim=9|id=646|"Boson,Site,n=1"), (dim=7|id=530|"Link,l=1"))
[2] ((dim=9|id=318|"Boson,Site,n=2")', (dim=9|id=318|"Boson,Site,n=2"), (dim=16|id=694|"Link,l=2"), (dim=7|id=530|"Link,l=1"))
[3] ((dim=9|id=818|"Boson,Site,n=3")', (dim=9|id=818|"Boson,Site,n=3"), (dim=16|id=596|"Link,l=3"), (dim=16|id=694|"Link,l=2"))
[4] ((dim=9|id=118|"Boson,Site,n=4")', (dim=9|id=118|"Boson,Site,n=4"), (dim=16|id=445|"Link,l=4"), (dim=16|id=596|"Link,l=3"))
[5] ((dim=9|id=452|"Boson,Site,n=5")', (dim=9|id=452|"Boson,Site,n=5"), (dim=8|id=927|"Link,l=5"), (dim=16|id=445|"Link,l=4"))
[6] ((dim=9|id=2|"Boson,Site,n=6")', (dim=9|id=2|"Boson,Site,n=6"), (dim=8|id=927|"Link,l=5"))


In [6]:
sites = [siteinds(H)[i][2] for i in 1:6]
@show sites

sites = Index{Int64}[(dim=9|id=646|"Boson,Site,n=1"), (dim=9|id=318|"Boson,Site,n=2"), (dim=9|id=818|"Boson,Site,n=3"), (dim=9|id=118|"Boson,Site,n=4"), (dim=9|id=452|"Boson,Site,n=5"), (dim=9|id=2|"Boson,Site,n=6")]


6-element Vector{Index{Int64}}:
 (dim=9|id=646|"Boson,Site,n=1")
 (dim=9|id=318|"Boson,Site,n=2")
 (dim=9|id=818|"Boson,Site,n=3")
 (dim=9|id=118|"Boson,Site,n=4")
 (dim=9|id=452|"Boson,Site,n=5")
 (dim=9|id=2|"Boson,Site,n=6")

In [7]:
psi0_init = randomMPS(sites,2)
@show psi0_init[1]

psi0_init[1] = ITensor ord=2
Dim 1: (dim=9|id=646|"Boson,Site,n=1")
Dim 2: (dim=2|id=687|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}
 9×2
  0.36385403344311107   0.25564167896201767
 -0.09234663690509878   0.402225520953778
  0.14791401968287562   0.10141145362300463
  0.21179780312656793   0.14050194361330615
  0.29918535221082465  -0.01742599737577917
  0.26674120793658324   0.335953791814634
 -0.11871516604757336   0.32491649942897255
 -0.22195848138167415  -0.028219034329818378
  0.27659801012036894   0.1229578470048066


ITensor ord=2 (dim=9|id=646|"Boson,Site,n=1") (dim=2|id=687|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}

In [8]:
nsweeps = 50 # 200
maxdim = [10,20,30] #,30,30,30, 40, 60, 80, 100]
# cutoff = [1E-10,1E-10,1E-10,1E-10,0.0]
cutoff = [1E-04,1E-05,1E-06,1E-06,1.E-07, 1.E-08, 1.E-09, 1.E-10, 1.E-12, 1.E-14]
noise = [1.e-06, 1.E-6, 1.E-06, 1.E-06, 0.0]

5-element Vector{Float64}:
 1.0e-6
 1.0e-6
 1.0e-6
 1.0e-6
 0.0

In [9]:
energy0, psi0 = dmrg([H],psi0_init; nsweeps, maxdim, cutoff, noise)

After sweep 1 energy=-0.3930931959439716  maxlinkdim=10 maxerr=7.15E-04 time=7.972
After sweep 2 energy=-1.5228413666017353  maxlinkdim=11 maxerr=8.97E-06 time=0.050
After sweep 3 energy=-1.8987570306797938  maxlinkdim=13 maxerr=8.20E-07 time=0.044
After sweep 4 energy=-1.9641485163985974  maxlinkdim=10 maxerr=8.62E-07 time=0.033
After sweep 5 energy=-1.9715783647852632  maxlinkdim=13 maxerr=8.79E-08 time=0.158
After sweep 6 energy=-1.971953039235366  maxlinkdim=16 maxerr=7.81E-09 time=0.040
After sweep 7 energy=-1.9719875885227893  maxlinkdim=21 maxerr=9.79E-10 time=0.047
After sweep 8 energy=-1.9719904713385796  maxlinkdim=30 maxerr=1.66E-10 time=0.066
After sweep 9 energy=-1.9719905351973548  maxlinkdim=30 maxerr=3.69E-11 time=0.428
After sweep 10 energy=-1.9719905354394887  maxlinkdim=30 maxerr=2.32E-11 time=0.205
After sweep 11 energy=-1.9719905354445197  maxlinkdim=30 maxerr=2.28E-11 time=0.142
After sweep 12 energy=-1.9719905354446237  maxlinkdim=30 maxerr=2.27E-11 time=0.141
Af

(-1.9719905354446272, MPS
[1] ((dim=9|id=646|"Boson,Site,n=1"), (dim=9|id=801|"Link,l=1"))
[2] ((dim=30|id=695|"Link,l=2"), (dim=9|id=318|"Boson,Site,n=2"), (dim=9|id=801|"Link,l=1"))
[3] ((dim=9|id=818|"Boson,Site,n=3"), (dim=30|id=447|"Link,l=3"), (dim=30|id=695|"Link,l=2"))
[4] ((dim=9|id=118|"Boson,Site,n=4"), (dim=30|id=673|"Link,l=4"), (dim=30|id=447|"Link,l=3"))
[5] ((dim=9|id=452|"Boson,Site,n=5"), (dim=9|id=651|"Link,l=5"), (dim=30|id=673|"Link,l=4"))
[6] ((dim=9|id=2|"Boson,Site,n=6"), (dim=9|id=651|"Link,l=5"))
)

In [10]:
y_shift = -3112.1604302407663  # eV (eq. position)
y_min = -3113.4044750979383  # eV (mean)
ZPE = energy0 * 27.21138 / 100 + y_shift - y_min
println("ZPE = $(ZPE) eV = $(ZPE * 2.194746E5 / 27.21138) cm-1") # to eV

ZPE = 0.707439019008234 eV = 5705.880985132859 cm-1


In [11]:
function sample_grid(psi)
    data = [[0 for _ in 1:9] for _ in 1:6]
    for i in 1:200
        grid = sample(psi)
        for (j, g) in enumerate(grid)
            data[j][g] += 1
        end
    end
    data
end

sample_grid (generic function with 1 method)

In [12]:
data = sample_grid(psi0)
@show data

data = [[0, 0, 13, 48, 72, 49, 17, 1, 0], [0, 0, 12, 46, 83, 47, 11, 1, 0], [1, 1, 13, 40, 76, 53, 15, 1, 0], [0, 1, 14, 56, 76, 45, 8, 0, 0], [0, 0, 11, 31, 74, 66, 18, 0, 0], [0, 0, 9, 35, 88, 58, 10, 0, 0]]


6-element Vector{Vector{Int64}}:
 [0, 0, 13, 48, 72, 49, 17, 1, 0]
 [0, 0, 12, 46, 83, 47, 11, 1, 0]
 [1, 1, 13, 40, 76, 53, 15, 1, 0]
 [0, 1, 14, 56, 76, 45, 8, 0, 0]
 [0, 0, 11, 31, 74, 66, 18, 0, 0]
 [0, 0, 9, 35, 88, 58, 10, 0, 0]

In [13]:
f = h5open("eigvecs.h5","w")
write(f, "psi$(0)", NDTensors.cpu(psi0))
close(f)

When you want to restart, use following code
```julia
f = h5open("eigvecs.h5","r")
psi0 = read(f, "psi$(0)", MPS)
close(f)
```

In [14]:
psis = [psi0]
@show psi0
energies = [energy0]

for i in 1:3 # 20
    psi_i_init = randomMPS(sites,2)
    #for j in 1:6
    #    psi_i_init[j] = to_float64_cuda_itensor(psi_i_init[j])
    #end
    local nsweeps = 50
    local maxdim = [10,20,30]#,30,30,30, 40, 60, 80, 100]
    local cutoff = [1E-04,1E-05,1E-06,1E-06,1.E-07, 1.E-08, 1.E-09, 1.E-10]
    local noise = [1.e-06, 1.E-6, 1.E-06, 1.E-06, 0.0]

    energy_i_tmp, psi_i_tmp = dmrg(H,psis,psi_i_init; nsweeps, maxdim, cutoff, noise)
    # continue while energy_i is not converged
    local maxdim = [30] #[100]
    local cutoff = [1.e-11, 1.E-12, 1.E-13, 1.E-14]
    local noise = [1.e-10, 1.E-10, 1.E-10, 1.E-10, 0.0]

    energy_i, psi_i = dmrg(H,psis,psi_i_tmp; nsweeps, maxdim, cutoff, noise)

    while abs(energy_i - energy_i_tmp) > 1E-11 #1E-12
        energy_i_tmp, psi_i_tmp = energy_i, psi_i
        energy_i, psi_i = dmrg(H,psis,psi_i_tmp; nsweeps, maxdim, cutoff, noise)
    end

    f = h5open("eigvecs.h5","w")
    write(f,"psi$(i)", NDTensors.cpu(psi_i))
    close(f)

    push!(psis, psi_i)
    push!(energies, energy_i)
    println("$(i)-th: $((energy_i - energy0)* 27.21138 / 100)") # to eV
    @show (energies .- energies[1]) .* 2.194746E5 ./100 # cm-1
end

@show (energies .- energies[1]) .* 2.194746E5 ./ 100# cm-1

psi0 = MPS
[1] ((dim=9|id=646|"Boson,Site,n=1"), (dim=9|id=801|"Link,l=1"))
[2] ((dim=30|id=695|"Link,l=2"), (dim=9|id=318|"Boson,Site,n=2"), (dim=9|id=801|"Link,l=1"))
[3] ((dim=9|id=818|"Boson,Site,n=3"), (dim=30|id=447|"Link,l=3"), (dim=30|id=695|"Link,l=2"))
[4] ((dim=9|id=118|"Boson,Site,n=4"), (dim=30|id=673|"Link,l=4"), (dim=30|id=447|"Link,l=3"))
[5] ((dim=9|id=452|"Boson,Site,n=5"), (dim=9|id=651|"Link,l=5"), (dim=30|id=673|"Link,l=4"))
[6] ((dim=9|id=2|"Boson,Site,n=6"), (dim=9|id=651|"Link,l=5"))

After sweep 1 energy=0.0962195250028637  maxlinkdim=9 maxerr=4.96E-04 time=0.157
After sweep 2 energy=-0.9447694711821043  maxlinkdim=11 maxerr=9.23E-06 time=0.041
After sweep 3 energy=-1.214526904988864  maxlinkdim=17 maxerr=9.15E-07 time=0.079
After sweep 4 energy=-1.283338725507353  maxlinkdim=14 maxerr=9.79E-07 time=0.063
After sweep 5 energy=-1.306530464297503  maxlinkdim=16 maxerr=8.77E-08 time=0.045
After sweep 6 energy=-1.3191710745521121  maxlinkdim=26 maxerr=7.64E-09 time

4-element Vector{Float64}:
    0.0
 1160.7148555416331
 1240.3839940403082
 1494.942244787389